In [18]:
import pickle
from urllib.parse import urlparse, parse_qs

In [2]:
import os
import pymongo
import pandas as pd
mongoPass = os.environ['mongoPass']
myclient = pymongo.MongoClient("mongodb+srv://axme100:{}@cluster0-5jopz.mongodb.net/test?retryWrites=true&w=majority".format(mongoPass))
mydb = myclient["finalProject"]
# Create a new colection called raw article
mycol = mydb["rawArticles"]
cursor = mycol.find()
raw_article_data = pd.DataFrame(list(cursor))

In [3]:
filename = 'url_terms'
infile = open(filename,'rb')
url_terms = pickle.load(infile)
infile.close()

In [15]:
url_terms

{'deportes-redes-sociales': 'deportes',
 'deportes': 'deportes',
 'sociedad': 'cultura-sociedad',
 'cultura': 'cultura-sociedad',
 'politica': 'política',
 'economia': 'economía',
 'portada': 'otro',
 'practicodeporte': 'deportes',
 'mundo': 'mundo',
 'gente': 'chisme',
 'destacada': 'otro',
 'efemotor': 'tecnología',
 'efefuturo': 'tecnología',
 'efeempresas': 'negocios',
 'efeestilo': 'moda',
 'mexico-empresas': 'méxico',
 'comunicados': 'otro',
 'reportajes': 'otro',
 'mexico': 'méxico',
 'efeverifica': 'otro',
 'efeemprende': 'otro',
 'efetur': 'otro',
 'nacional': 'méxico',
 'comunidad': 'cultura-sociedad',
 'adrenalina': 'deportes',
 'global': 'mundo',
 'funcion': 'otro',
 'opinion': 'otro',
 'trending': 'chisme',
 'expresiones': 'chisme',
 'hacker': 'tecnología',
 'multimedia': 'otro',
 'tecnologia': 'tecnología',
 'estados': 'méxico',
 'capital': 'negocios',
 'espectaculos': 'otro',
 'index.php': 'otro',
 'ciencias': 'tecnología',
 'elpais': 'otro',
 'internacional': 'mundo',
 

In [4]:
raw_article_data.head()

,_id,url,title,date,publication,articleText,authors,errors
0,5e4eed7d5b0ac2de38a07b19,https://www.excelsior.com.mx/adrenalina/chucky...,‘Chucky’ Lozano no está al cien,2020-02-20 20:32:35,Excelsior,"NÁPOLES, Ita.El técnico del Nápoles, Gennar...",arturo.duran,[]
1,5e4eed7d5b0ac2de38a07b1a,https://www.excelsior.com.mx/comunidad/cuidado...,¡Cuidado! Así aumentó el robo de menores en la...,2020-02-20 20:32:20,Excelsior,"CIUDAD DE MÉXICO Durante 2019, la Fiscalía G...",blanca.betan,[]
2,5e4eed7e5b0ac2de38a07b1b,https://www.excelsior.com.mx/funcion/se-virali...,Se viraliza video inédito de Rammstein tocando...,2020-02-20 20:31:12,Excelsior,"CIUDAD DE MÉXICOSi te gusta el metal, y eres ...",magnolia.ramirez,[]
3,5e4eed7e5b0ac2de38a07b1c,https://www.excelsior.com.mx/global/conoce-a-u...,¡Conoce a una verdadera Reina del carnaval de ...,2020-02-20 20:29:08,Excelsior,"RÍO DE JANEIRO, BRASIL. Con su sonrisa radian...",manuel.velazquez,[]
4,5e4eed7f5b0ac2de38a07b1d,https://www.excelsior.com.mx/comunidad/dicen-a...,Dicen aquí que transexuales tienen 'trastorno ...,2020-02-20 20:27:22,Excelsior,CIUDAD DE MÉXICOEn el Foro “Infancias Trans: ...,blanca.betan,[]


In [25]:
def get_category_from_url(url, publication):
    '''Input: URL and Publication
       Output: Discrete category
    '''
    
    # First create a dictionary that will check to see
    # If we can just get the category from the XML term
    xmlCategories = {'Expansión - Emprendedores': 'negocio',
                    'Expansión - Nacional': 'méxico',
                    'Expansión - Dinero': 'negocio',
                    'Expansión - Carrera': 'negocio',
                    'Expansión - Opinión': 'opinión',
                    'Expansión - Tecnología': 'tecnología',
                    'Expansión - Empresas': 'negocio'}
    
    # First check to see if the publication is in the dictionary
    # That we have created above, if it is then just return it
    if publication in xmlCategories:
        return xmlCategories[publication]
    
    # In case it is not, then we need to get the category from the URL
    else:
        parsed_url = urlparse(url)
        parsed = parsed_url.path
        potential_paths = parsed.split('/')
        
        for path in potential_paths:
            if path in url_terms:
                return url_terms[path]
            else:
                pass
        
        return "otro"
    

In [26]:
# Create a new column called cateogry applying the function
raw_article_data['category'] = raw_article_data.apply(lambda x: get_category_from_url(x.url, x.publication), axis=1)

In [27]:
# Do a little bit of analyis to make sure that it works safely

In [28]:
raw_article_data['category'].value_counts()

deportes            1425
otro                1410
cultura-sociedad    1404
méxico               784
política             639
mundo                605
economía             439
chisme               205
negocio              182
tecnología           159
negocios              95
opinión               45
viajes                17
moda                  14
Name: category, dtype: int64

In [11]:
len(raw_article_data)

7423